# Welcome to the ProcessMCRaT Example Notebook


### This notebook will walk through some of the ways that the ProcessMCRaT library can be used to process the output of a [MCRaT](https://github.com/lazzati-astro/MCRaT) simulation and conviently get and plot results quickly.

This notebook can be run interactively on your local machine if you clone the git repository with 
```
git clone https://github.com/parsotat/ProcessMCRaT.git
```
After, the notebook can be accessed by navigating to the `notebook/` directory and running 
```
jupyter notebook
```
in the command line.

Alternatively, this notebook can be run remotely and interactively by opening it in Binder at the link: [It is possible to include hyperlinks](https://www.example.com)

In either case the steps below will help you get an understanding of how to setup and use the ProcessMCRaT python package.

#### 1. First we need to install the package which can be done using *pip*

In the command line you would run `pip install processmcrat` or from a jupyter notebook you would run the line below:

In [ ]:
!pip install processmcrat

#### 2. With the package now installed on your local machine or in the Binder environment we can now import the package alongside other packages that we will need to analyze a given MCRaT simulation

In [7]:
import processmcrat as pm
import astropy.units as unit
from astropy import constants as const
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
plt.ion() #this allows matplotlib plots to show without explicit calls to plt.show

#### 3. Now that we have loaded everything, we need a MCRaT simulation file to analyze. I have placed one such file in this directory from the variable *40sp_down* simulation that includes polarization and cyclo-synchrotron emission and absorption. The simulation file correponds to the last frame of the hydrodynamic simulation and we will produce mock observations from it.

We first need to create a MCRaT simulation load object that will hold all the information in the mcdata file. We pass the location of the directory that holds the simulation's mcdata files. If we dont pass any file directory then the library assumes that the mcdata file(s) are in the current working directory.

In [5]:
mcrat_sim=pm.McratSimLoad() 

if the mcdata files are located in `/Volume/dir/to/mcrat/sim`, we would instead do `pm.McratSimLoad('/Volume/dir/to/mcrat/sim')`  

Now we need to specify which frame we want to load and whether we want to load any other optional data such as stokes parameters, comoving four momenta, or the photon type. For our example analysis, the last frame of the hydro simulation is frame 1999 which is the last frame of the MCRaT simulation as well. Additionally, we only want to load in the stokes parameters to calculate polarizations later on. By leaving out the `read_comv` and `read_type` keywords, they are assumed to be `False`.

In [6]:
mcrat_sim.load_frame(1999, read_stokes=True)

The `mcrat_sim` object now holds all the information for all the MCRaT photons and various quantities can be accessed of desired. We can get one of the following quantities: 
* lab frame 4 momenta (*p0, p1, p2, p3*), 
* location of each photon in cartesian coordinates of x,y, and z (*r0, r1, r2*), 
* weights (*weight*), 
* number of scatterings experienced by each photon (*scatterings*), 
* location of the photon in the original hdf5 mcdata file (*file_index*)
* comoving fluid frame 4 momenta if loaded (*comv_p0, comv_p1, comv_p2, comv_p3*),
* photon type if loaded (*photon_type*)
* and the stokes parameters if loaded (*s0, s1, s2, s3*)

The quantities are accessed from the `mcrat_sim` object as `mcrat_sim.loaded_photons.x`, where x is any of the italicised identifiers listed above. For example, if we want to see the photon energies we can do:

In [9]:
mcrat_sim.loaded_photons.p0*(const.c.cgs.value)

array([5.27059383e-08, 1.26476577e-08, 6.22665414e-09, ...,
       2.00832919e-12, 2.74379951e-12, 6.48917431e-13])

Here we see that the loaded photons' 4-momenta are normalized by c, the speed of light in cgs units, which is how they are stored in the MCRaT simulations. We can get thier energies in erg simply by multiplying by c.

Since the energies are something that is typically analyzed, the `mcrat_sim.loaded_photons`, which is an instance of the PhotonList class, has a method associated with it that allows us to quickly calculate the energies of the photons. It can be called as:

In [10]:
mcrat_sim.loaded_photons.get_energies() #for comoving energies of the photons, the function is: get_comv_energies()

array([3.28964592e+01, 7.89404701e+00, 3.88637183e+00, ...,
       1.25350049e-03, 1.71254496e-03, 4.05022403e-04])

By default, the energy is calculated in units of keV, but by passing an applicable astropy unit to the function, the energy will be appropriately calculated. For example, to calculate the units in ergs, we do

In [11]:
mcrat_sim.loaded_photons.get_energies(unit=unit.erg)

array([5.27059383e-08, 1.26476577e-08, 6.22665414e-09, ...,
       2.00832919e-12, 2.74379951e-12, 6.48917431e-13])

If we want a wavelength, we can also do that.

In [12]:
mcrat_sim.loaded_photons.get_energies(unit=unit.angstrom)

array([1.04697493e-15, 2.51238872e-16, 1.23689113e-16, ...,
       3.98943720e-20, 5.45040918e-20, 1.28903934e-20])

#### An aside about units

This brings us to an important aside about units used in ProcessMCRaT. The package assigns appropriate astropy units to various quantities to make sure that various calculations are done consistently. It is beneficial for the user to read some of the excellent documentation that the astropy collaboration has on their implementation of units as it will help aid in understanding the ProcessMCRaT library. This documentation can be found at [https://docs.astropy.org/en/stable/units/](https://docs.astropy.org/en/stable/units/)

#### 4. With a basic understanding of the McratSimLoad object and its use in loading in simulation data, we can create a mock observation with the photons. To do that we create a MockObservation object. This object, holds all the data for:
* where an observer is located with respect to the jet axis, $\theta_\mathrm{v}$, 
* the acceptance angle of the observer $\Delta \theta$, meaning that photons that are moving towards the observer in the range [$\theta_\mathrm{v}-0.5\Delta \theta, \theta_\mathrm{v}+0.5\Delta \theta$) are considered to be detected
* the location of the observer, $r_\mathrm{d}$, 
* the number of frames per second in the hydrodynamic simulation that was analyzed, $\mathrm{fps}$,
* the number of dimensions of the hydrodynamic simulation, and
* all of the data for photons detected by the observer at $\theta_\mathrm{v}$

For the *40sp_down* simulation, we would make a mock observation as:

In [13]:
observation=pm.MockObservation(1, 1, 1e13, 10, mcratsimload_obj=mcrat_sim)

where the first variable passed to the observation function is $\theta_\mathrm{v}$ in degrees, the second is $\Delta \theta$ in degrees, the third is $r_\mathrm{d}$ in cm, the fourth is fps. The last argument is the McratSimLoad object that contains all the loaded photons' data. We did not specify the number of dimensions in the call because the default is set to 2D. If the hydro simulation was in 3D, which will be supported in the future, you would have `hydrosim_dim=3` keyword as one of the arguments. 

The mock observation function call calculates which photons are moving towards the observer in the angle range specified previously. For these detected photons, their detection time, $t_d$, is calculated as:
$t_d=t_\mathrm{real} + t_p - t_j$, where $t_\mathrm{real}$ is the lab frame time of the hydrodynamic frame that has been loaded, $t_j$ is the jet detection time which is calculated by considering a virtual photon propagating to $r_\mathrm{d}$ at the time that the jet is initially launched. Finally, $t_p$ is the time that it will take photons to propagate towards $r_\mathrm{d}$ from their location in the loaded MCRaT frame.

Like the McratSimLoad object that we were exploring earlier, the MockObservation object that we created contains all the information for the detected photons. We can get all the same photon data as before with the addition of each photons' detection time. To access this data we would do:

In [15]:
observation.detected_photons.p0

array([1.10302427e-18, 1.17673392e-18, 6.61093762e-19, ...,
       5.72090955e-22, 3.51041574e-22, 1.38124376e-22])

In [14]:
observation.detected_photons.detection_time

array([11.70556666, 11.75620169, 11.68942642, ..., 39.33796243,
       39.32549413, 39.44886216])

In order to easily save our observations, we can create event files. These event files only contain information about the detected photons at the user specified $\theta_\mathrm{v}$ which allow them to be smaller than the MCRaT mcdata files that can be very large and difficult to send to collaborators, for example. These files can be loaded as an observation object, allowing for continued analysis of the data. 

In order to create an event file, we can do:

In [16]:
observation.save_event_file('40sp_down')

NameError: name 'create_obs_id' is not defined